## Assignment 3: Machine Translation/Sequence to Sequence Model

#### Overview and Motivation
In assignment 1 and 2, we learned how to use dense nets and conv nets to perform classification tasks. In this final assignment, we will learn how to deal with the sequential data. Such data is ubiquitous in healthcare -- think EKG, EEG, vital stats over time or DNA/RNA sequences. To get comfortable with sequence models, we will implement a sequence-to-sequence (seq-to-seq) model for data _translation_. 

The goal is the following: given two sequences $X = (x_1,x_2,\ldots,x_{N_i})$ and $Y = (y_1,y_2,\ldots,y_{M_i})$, we would like our neural net $f_\theta(\cdot)$ to map or "translate" $x$ to $y$. Given what we have learned so far, given a training dataset $\{(X_i,Y_i)\}_{i=1,\ldots,S}$, a first approach we might think of is to take $X_i$ as an input vector, and $Y_i$ as an output vector, and train a dense or conv net to minimize some loss between input data $X_i$ and the translated data $Y_i$. 

To be more specific, let's think about the problem of language translation, where for each training data point $(X_i, Y_i)$, where $X_i$ is a sentence in English and $Y_i$ is a sentence in French. What is the problem with our approach of using a dense net? If we think about it, each input English sentence can be of different length, additionally, the translated output sentence could vary in length. For previous assignments, we could easily resize our images to be of the same size while preserving some structure of the original image, however, we cannot "resize" all English sentences to a corresponding English sentence that preserves its semantic meaning. Succinctly, the issue is that _a priori_ we do not know the lengths of $X_i$ or $Y_i$. _Sequence models with RNNs help us solve this problem!_

#### High level overview
As humans, when we translate between languages, we don't look at each word and literally translate each word from source to target language -- the context matters! We read the sentence, understand it, then we translate it. More formally, we can think of the translation process as the following: we map a sentence from source space $\mathcal{E}$ (space of all English sentences), to some hidden space (latent space e.g. $\mathcal{H} \subseteq \mathbb{R}^n$), and finally to destination space $\mathcal{F}$ (space of all French sentences). Now our goal is to learn a function $f: \mathcal{E} \rightarrow \mathcal{F}$, that maps sentences from English to French. We can decompose this function $f$ into composition of two neural nets, $f = d \circ e$. The encoder neural net, $e: \mathcal{E} \rightarrow \mathcal{H}$, _encodes_ English to a latent vector representation, and the decoder neural net $d: \mathcal{H} \rightarrow \mathcal{S}$, _decodes_ the latent vector to French. 
 

We will model the encoder $e(\cdot)$ and the decoder $f(\cdot)$ as LSTMs. We can then visualize our translation process by "unrolling" the RNN (see figure below). The encoder LSTM takes the input sequence, one word at a time, then performs the forward computation to output a latent vector. The decoder LSTM then takes this latent vector and the previous word in the target sentence as an input, and maps it to distribution over the vocabulary of the destination language (these are the possible candidates for the next word). 

What is the previous word input to the decoder LSTM when no first word has been generated? We use a dummy start sentence token for this purpose. Similarly, when do we end translating the output sentence? when we output an end sentence token. The start and end tokens are appended to each sentence during data preprocessing. 

<img src="./img/seq_to_seq_io.png" width="80%">


Lastly, since we need to perform operations on sentences, we need to somehow represent them as vectors. One simple way of doing this is to first compute a vocabulary for the language -- this could be done, for example be picking the top 10,000 words. We assign all these words an id from 0 to 9,999, to get a unique integer id for each word. Then given a sentence, we can map each word in the sentence to its id, to get a vector representation of the sentence. For example, the sentence "The dog jumped over the fence" might be mapped to `[200, 140, 672, 243, 200, 997]`. These numbers can be represented as vectors by mapping them to a [one-hot-vector](https://en.wikipedia.org/wiki/One-hot). Think about why this is not optimal.

This is not optimal because with this encoding scheme, the words "good", "bad", "great" are equally far apart in (Euclidean) distance i.e. the distance between the one-hot-vector for the words "good", "bad", "great" are equally far apart. Instead, we would like to map these words to vector representations that preserve some semantic meaning of the words. A popular way to do this is to use [word embeddings](https://en.wikipedia.org/wiki/Word_embedding). The idea is that the vector representing the word captures the semantic closeness between the words. For example, we would like the words "king" and "queen", and "walk" and "walking" to be close in this embedding space. 

<img src="./img/word_vectors.png" width="80%">


With this high-level picture in mind let's think about how we should build the encoder and decoder nets, and how we can setup an appropriate loss.  

#### Let's start thinking about how we can mathematically model this process. 

Input sentence: $X_i = (x_1,x_2,\ldots,x_{N_i})$  
Output sentence: $Y_i = (y_1,y_2,\ldots,y_{M_i})$  
Latent vector (["thought vector"](https://drive.google.com/file/d/0B8i61jl8OE3XdHRCSkV1VFNqTWc/view)): $v$   
Source vocabulary: $V_s = \{s_i\}_{i=1,\ldots,P} $  
Destination vocabulary: $V_d = \{d_i\}_{i=1,\ldots,P}$

We model the probability of a destination sentence given source destination i.e. $p(Y_i | X_i)$ as:

$$p(Y_i | X_i) = p(y_1,y_2,\ldots,y_{M_i} | v, x_1,x_2,\ldots,x_{N_i}) = \prod_{m=1}^{M_i} p(y_m | v, y_1,y_2,\ldots,y_{m-1})$$

Here, we have assumed:  
1. $Y_i$ is [_conditionally_ independent](https://en.wikipedia.org/wiki/Conditional_independence) of $X_i$ given the abstract representation vector $v$   
2. The probability of a word $y_m$ only depends on the words preceding it $ (y_1,y_2,\ldots,y_{m-1}) $.   

The goal is then to learn the model parameters by maximizing the likelihood for this model. Or equivalently, minimizing the cross-entropy between the predicted target sequence and the actual target sequence.

#### Implementation Steps
At this point in the course, we have all the tools to implement widely popular deep learning model which was the basis for [Google's state of the art results on language translation](https://ai.googleblog.com/2016/09/a-neural-network-for-machine.html). To get comfortable with reading and understanding a deep learning research paper, in this assignment we will implement the 2014 paper from Sutskever et al. [Sequence to Sequence Learning
with Neural Networks](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf). 

Okay so how do we actually code this up? The key is to recognize the parts that you have already seen. Let's start with listing what we need.

a. Pre-requisites:
   1. Read and understand the [seq-to-seq paper](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf).
   2. If you do not have access to GPUs -- you should use Google collab with GPU/TPU (Read more [here](https://colab.research.google.com/notebooks/gpu.ipynb))


b. Data: start by downloading the [English to French sentence pairs](http://www.manythings.org/anki/).  
   
c. Source and destination vocabularies: 
   * The first thing we need to do is build the vocabulary for the source and destination languages. To do this, we first need to do some data prep -- you should follow and adapt the steps under section "download_and_prepare_the_dataset" [here](https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention#download_and_prepare_the_dataset)
   
d. The input and output sequences represented as vectors of numbers. 
   * Once you have the vocabularies for English and French, you would like to represent your source and target sentences as a vector. If you followed, steps above correctly, the tensor `input_tensor` and `target_tensor` will contain the vector representations of the sentences from the training data. To make sure you have performed this correctly, map some samples from both the `input_tensor` and `target_tensor` back to their word-sentence representations and print them.
   * You should get `load_dataset(path, num_examples=None)` function to work for the Engligh-French dataset. 
   * Create `tf.dataset`, you should be familiar with this now.
   
e. Function to map source sentence to the latent vector.  

Implement `class Encoder(tf.keras.Model):` you may use the tf tutorial Encoder class as a guide, however, note our model is different. If you understand what each part of this class is doing, adapting it for LSTM should be fairly straightforward. 
   1. Build an encoder neural net. To map the input sentence to its embedding, you can use the Keras embedding layer https://keras.io/layers/embeddings/
   2. The embedding is the input to an encoder LSTM, you may use the Keras [LSTM layer](https://keras.io/layers/recurrent/)
   3. The output of this LSTM is the hidden state as well as the cell state (use `return_state=True` for the encoder LSTM)
   
f. Function to map latent vector to destination sentence.

Implement `class Decoder(tf.keras.Model):`
   1. You may use the Keras embedding layer to compute the destination sentence embeddings.
   2. Create a decoder LSTM, set the initial cell state to the cell state output of the encoder LSTM. With `return_state=False` and `return_sequences=True`, this LSTM will output a sequence, where each element is the output at a given time step. 
   3. Use a dense softmax layer to map each output element of the sequence to a distribution over the target vocabulary.
   
g. Training: The loss is simply the cross entropy loss between the input target sentence and output target sequence. 
   1. Write a loss function `def loss_function(real, pred):` which computes this loss.
   

#### Practical Advice 
a  As always split your data into training and testing.  
b. Before you try training the entire model on a large portion of your data. Pick a few sentences ~10,20,30, make sure you can over fit your model. That is, make sure you can translate them from source to destination. This is always a good sanity check when implementing deep learning models. If your model cannot learn to map a few sentences by overfitting to the small train set then it is unlikely to work on a larger dataset.   
c. Use the following hyperparams:  
   * batch_size = 64   
   * epochs = 100  
   * latent_dim = 256  
   * units = 1024
   * num_samples = 10000  

#### Results   
a. Plot your train and test loss.  
b. Recreate Fig. 2 (Sutskever et al.) for this dataset, you may use [scikit-learn PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) for this task. 
c. Generate and create a table of translations, similar to Table 3. (Sutskever et al.) 
   
_Note: implementing target sentence reversal used in the Sutskever is not required, but might help your model perform better_

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

#!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time